In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import sqlite3
import pandas as pd
import numpy as np
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

TensorFlow 1.x selected.


In [2]:
!test -f newsticker2019.db || wget https://datanizing.com/bert/heise/newsticker2019.db

--2020-07-17 08:58:41--  https://datanizing.com/bert/heise/newsticker2019.db
Resolving datanizing.com (datanizing.com)... 37.221.195.1
Connecting to datanizing.com (datanizing.com)|37.221.195.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1183744 (1.1M) [application/octet-stream]
Saving to: ‘newsticker2019.db’

newsticker2019.db   100%[===================>]   1.13M  1.94MB/s    in 0.6s    

2020-07-17 08:58:42 (1.94 MB/s) - ‘newsticker2019.db’ saved [1183744/1183744]



In [4]:
sql = sqlite3.connect("newsticker2019.db")
texts = pd.read_sql("SELECT id, text FROM news", sql, index_col="id")

In [5]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [6]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [7]:
from tqdm.auto import trange
vecs = []
for i in trange(int(len(texts)/100)+1):
    r = elmo_vectors(texts[i*100:(i+1)*100].text.map(str).values)
    if len(vecs) > 0:
        vecs = np.vstack([vecs, r])
    else:
        vecs = r

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
# Cosinus-Ähnlichkeit aller Dokumente miteinander berechnen
sim = cosine_similarity(vecs, vecs)

In [9]:
# Ergebnis ist eine Matrix
sim.shape

(10077, 10077)

In [10]:
# Größter Wert wird für identische Meldungen gefunden
sim.max()

1.0000008

In [11]:
for threshhold in [0.7, 0.8, 0.9, 0.95]:
    s = sim.copy()
    # alle über dem Threshhold ausmaskieren
    s[s > threshhold] = 0
    (max_a, max_b) = np.unravel_index(np.argmax(s), s.shape)
    print("%f '%s' '%s'" % (threshhold, texts.iloc[max_a]["text"], texts.iloc[max_b]["text"]))

0.700000 'Virenschutz im Test: Reicht der Windows Defender wirklich?' 'Mars-Rover Curiosity: Zwei Sonnenverfinsterungen in wenigen Tagen'
0.800000 'Digitalradio: Österreich bekommt Ende Mai zusätzliche Sendestationen' 'Strom speichern mit Wasser: Potenzial für Pumpspeicherkraftwerke hundertmal größer als Bedarf'
0.900000 'Verteidigung: EU-Parlament gibt Milliarden frei, aber nicht für Killer-Roboter' 'Bundesrat: E-Scooter dürfen nicht auf Gehwege und erst ab 14 genutzt werden'
0.950000 'IT-Jobtag in Hamburg: Arbeitgeber treffen am 27. Februar auf Bewerber' 'IT-Jobtag in Düsseldorf: Arbeitgeber treffen am 21. März auf Bewerber'


In [12]:
# Alternative Aufgabe: Ähnlichste Nachrichten in unterschiedlichen Monaten finden